In [16]:
%matplotlib notebook
from matplotlib import pyplot as plt
import mpl_finance as finance
import pandas as pd
import numpy as np
from datetime import datetime
from pymongo import MongoClient
from DCE import generalize as DCE_READ
from CFFEX import generalize as CFFEX_READ
from SHFE import generalize as SHFE_READ
from CZCE import generalize as CZCE_READ
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cherrypy

In [17]:
client = MongoClient('mongodb://localhost:27017/')
db = client.futures

In [31]:
a = SHFE_READ(db.shves.find({ 'DATE': "20180705" }))[['ClosePrice', 'Volume', 'OpenInterest']]
b = CFFEX_READ(db.cffexes.find({ 'tradingday': "20180705" }))[['ClosePrice', 'Volume', 'OpenInterest']]
c = DCE_READ(db.dces.find({ 'date': "180704" }))[['ClosePrice', 'Volume', 'OpenInterest']]
d = CZCE_READ(db.czces.find({ 'date': "180705" }))[['ClosePrice', 'Volume', 'OpenInterest']]

In [19]:
@interact(Market='CZCE', Name='AP', Month='807')
def f(Market, Name, Month):
    market, name, month = Market, Name, Month
    data = None
    try:
        if market == 'SHFE':
            data = SHFE_READ(db.shves.find({ 'PRODUCTNAME': name, 'DELIVERYMONTH': month }))
        elif market == 'CFFEX':
            data = CFFEX_READ(db.cffexes.find({ 'instrumentid': name + month }))
        elif market == 'DCE':
            data = DCE_READ(df.dces.find({ 'instrumentname': name, 'instrumentmonth': month }))
        elif market == 'CZCE':
            data = CZCE_READ(db.czces.find({ 'instrumentid': name + month }))
        print('%s %s%s -> %d entries' % (market, name, month, len(data)))
        if len(data) > 0:
            data.set_index('TradingDay', inplace=True)
            w = widgets.SelectMultiple(
                options=data.columns,
                value=['OpenPrice'],
                description='Fruits',
                disabled=False
            )
            fig, ax = plt.subplots()
            plt.title("%s %s%s" % (market, name, month))
            def on_change(v):
                ax.clear()
                data[list(w.value)].plot(ax=ax)
            w.observe(on_change, names='value')
            display(w)
    except:
        print('Error')


interactive(children=(Text(value='CZCE', description='Market'), Text(value='AP', description='Name'), Text(val…

In [30]:
@interact(Market='CZCE', Name='AP', Month='807')
def f(Market, Name, Month):
    market, name, month = Market, Name, Month
    data = None
    try:
        if market == 'SHFE':
            data = SHFE_READ(db.shves.find({ 'PRODUCTNAME': name, 'DELIVERYMONTH': month }))
        elif market == 'CFFEX':
            data = CFFEX_READ(db.cffexes.find({ 'instrumentid': name + month }))
        elif market == 'DCE':
            data = DCE_READ(df.dces.find({ 'instrumentname': name, 'instrumentmonth': month }))
        elif market == 'CZCE':
            data = CZCE_READ(db.czces.find({ 'instrumentid': name + month }))
        print('%s %s%s -> %d entries' % (market, name, month, len(data)))
        if len(data) > 0:
            data.set_index('TradingDay', inplace=True)
            fig, ax = plt.subplots()
            plt.title("%s %s%s" % (market, name, month))
            data = data[data['OpenPrice'] != 0]
            finance.candlestick2_ochl(ax, data['OpenPrice'], data['ClosePrice'], data['HighestPrice'], data['LowestPrice'], width=1)
            plt.show()
    except:
        print('Error')


interactive(children=(Text(value='CZCE', description='Market'), Text(value='AP', description='Name'), Text(val…